In [1]:
!pip install rapidfuzz


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import os, sys, pandas as pd
from pyspark.sql import SparkSession

def log(msg): print(msg, file=sys.stdout, flush=True)

In [3]:
print('helloworltrdf')

helloworltrdf


In [4]:
# ── 1. Spark session ────────────────────────────────────────────────
spark = SparkSession.getActiveSession()
if spark is None:
    spark = (
        SparkSession.builder
        .appName("ReloadBronzeAug2021")
        .config("spark.driver.memory", "8g")
        .getOrCreate()
    )
spark.conf.set("spark.sql.debug.maxToStringFields", 200)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/16 17:13:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:

# ── 2. Locate bronze files (try "2021-8" then "2021-08") ────────────
bronze_dir = os.path.join(os.getcwd(), "datamart", "bronze")
snapshot   = "2021-8"                     # default pattern

paths = {
    "resume": os.path.join(bronze_dir, f"resume_{snapshot}.parquet"),
    "jd":     os.path.join(bronze_dir, f"jd_{snapshot}.parquet"),
    "labels": os.path.join(bronze_dir, f"labels_{snapshot}.parquet"),
}

# fallback to zero-padded filenames
if not all(os.path.exists(p) for p in paths.values()):
    snapshot = "2021-08"
    paths = {k: os.path.join(bronze_dir, f"{k}_{snapshot}.parquet")
             for k in ["resume", "jd", "labels"]}

# ensure every file exists
for name, p in paths.items():
    assert os.path.exists(p), f"{name.capitalize()} file missing: {p}"


In [6]:
# ── 3. Load Parquet shards ──────────────────────────────────────────
resume_df = spark.read.parquet(paths["resume"])
jd_df     = spark.read.parquet(paths["jd"])
labels_df = spark.read.parquet(paths["labels"])

log(f"Loaded RESUME  → rows: {resume_df.count():,}")
log(f"Loaded JD      → rows: {jd_df.count():,}")
log(f"Loaded LABELS  → rows: {labels_df.count():,}")

Loaded RESUME  → rows: 1,038
Loaded JD      → rows: 1,038
Loaded LABELS  → rows: 1,038


In [7]:
# ── 4. Show schemas & small samples ─────────────────────────────────
pd.set_option("display.max_colwidth", 250)

log("\n========== RESUME SCHEMA ==========")
resume_df.printSchema()

log("\n--- Nested `education` schema (resume) ---")
resume_df.select("education").printSchema()

log("\n--- Sample resume `education` arrays & `certifications` ---")
display(
    resume_df
      .select("id", "education", "certifications")
      .limit(5)
      .toPandas()
)

log("\n========== JD SCHEMA ==========")
jd_df.printSchema()

log("\n--- Sample JD `required_education` & `certifications` ---")
display(
    jd_df
      .select("id", "required_education", "certifications")
      .limit(5)
      .toPandas()
)

log("\n========== LABELS SCHEMA ==========")
labels_df.printSchema()

log("\n--- Sample label rows ---")
display(
    labels_df
      .limit(5)
      .toPandas()
)



========== RESUME SCHEMA ==========
root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- location_preference: string (nullable = true)
 |-- work_authorization: string (nullable = true)
 |-- employment_type_preference: string (nullable = true)
 |-- hard_skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- soft_skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- languages: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- experience: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- role: string (nullable = true)
 |    |    |-- company: string (nullable = true)
 |    |    |-- date_start: string (nullable = true)
 |    |    |-- date_end: string (nullable = true)
 |    |    |-- role_description: string (nullable = true)
 |    |    |-- snapshot_date: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |-- education: ar

,id,education,certifications
0,RES_CHUA6p90,"[(Bachelor of Science, California State University Northridge, None, None, 3.6500000953674316, Healthcare Administration. Additional Coursework in Healthcare Leadership, Economy, Geriatric Care, Public Health, Biostatistics, Legal Health, IT and ...",[]
1,RES_SlLMkiqd,"[(Doctor of Public Health, George Washington University, Milken Institute School of Public Health, None, present, None, None, None, None), (Master of Science: Public Health, Tulane University School of Public Health, None, 2012-05-01T00:00:00, No...",[]
2,RES_mFb6f8iE,[],[]
3,RES_zkuNDo8d,"[(Bachelor of Technology, Mahatma Gandhi University, None, 2006-01-01T00:00:00, None, Computer Science and Engineering, None, None), (B.Tech - CSE, Mahatma Gandhi University, None, 2006-01-01T00:00:00, None, None, None, None), (None, Mahatma Gand...",[]
4,RES_PqbOkeex,[],[]



========== JD SCHEMA ==========
root
 |-- id: string (nullable = true)
 |-- company_name: string (nullable = true)
 |-- role_title: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_location: string (nullable = true)
 |-- about_the_company: string (nullable = true)
 |-- job_responsibilities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- required_hard_skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- required_soft_skills: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- required_language_proficiencies: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- required_education: string (nullable = true)
 |-- required_work_authorization: string (nullable = true)
 |-- certifications: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- snapshot_date: date (nullable = true)


--- Sample JD `required_education` & `certificat

,id,required_education,certifications
0,JD_CHUA6p90,None,[]
1,JD_SlLMkiqd,Bachelor's degree in computer science or a related discipline required. Master's degree preferred.,[]
2,JD_mFb6f8iE,"BS or BA with emphasis in computer science, information systems, or business administration","[Certification of Competency in Business Analysis (CCBA), Certified Business Analysis Professional (CBAP)]"
3,JD_zkuNDo8d,"A bachelor's degree in IT, Computer Science or related field or an MBA.",[]
4,JD_PqbOkeex,Bachelor's Degree and six to ten years of experience or equivalent education and software engineering training or experience,"[AWS, Azure]"



========== LABELS SCHEMA ==========
root
 |-- _id: string (nullable = true)
 |-- resume_id: string (nullable = true)
 |-- job_id: string (nullable = true)
 |-- fit: string (nullable = true)
 |-- snapshot_date: date (nullable = true)
 |-- fit_score: double (nullable = true)


--- Sample label rows ---


,_id,resume_id,job_id,fit,snapshot_date,fit_score
0,684547f570634603a66d8df5,RES_CHUA6p90,JD_CHUA6p90,No Fit,2021-08-11,0.0
1,684547f870634603a66d8df7,RES_SlLMkiqd,JD_SlLMkiqd,No Fit,2021-08-26,0.0
2,6845480470634603a66d8dfe,RES_mFb6f8iE,JD_mFb6f8iE,No Fit,2021-08-09,0.0
3,6845481670634603a66d8e09,RES_zkuNDo8d,JD_zkuNDo8d,No Fit,2021-08-09,0.0
4,6845481a70634603a66d8e0c,RES_PqbOkeex,JD_PqbOkeex,No Fit,2021-08-21,0.0


In [8]:
from pyspark.sql.functions import size, col
import pandas as pd

# 1) Total rows
total = resume_df.count()

# 2) Count empty arrays
empty_count = (
    resume_df
    .filter(size(col("education")) == 0)
    .count()
)

# 3) Calculate percentage
empty_pct = (empty_count / total * 100) if total else 0.0

# 4) Build a tiny pandas report
report = pd.DataFrame([{
    "column": "education",
    "empty_array_count": empty_count,
    "empty_array_pct": round(empty_pct, 2)
}])

print(f"Total rows in resume_df: {total:,}")
display(report)


Total rows in resume_df: 1,038


,column,empty_array_count,empty_array_pct
0,education,80,7.71


In [9]:
from pyspark.sql.functions import size, col, trim, isnan
import sys

def log(msg):
    print(msg, file=sys.stdout, flush=True)

# ── RESUME_DF METRICS ─────────────────────────────────────────────────────────
total_resume = resume_df.count()

# 1) % of resume_df.education arrays with length < 3
ed_lt3_count = resume_df.filter(size(col("education")) < 3).count()
ed_lt3_pct   = (ed_lt3_count / total_resume * 100) if total_resume else 0.0
log(f"Total resume_df rows:                        {total_resume:,}")
log(f"education length < 3:                        {ed_lt3_count:,} ({ed_lt3_pct:.2f}%)")

# 2) % of resume_df.certifications arrays with length == 0
cert_empty_count = resume_df.filter(size(col("certifications")) == 0).count()
cert_empty_pct   = (cert_empty_count / total_resume * 100) if total_resume else 0.0
log(f"certifications length == 0:                  {cert_empty_count:,} ({cert_empty_pct:.2f}%)")


# ── JD_DF METRICS ────────────────────────────────────────────────────────────
total_jd = jd_df.count()

# 3) % of jd_df.required_education strings that are NULL or empty
req_ed_missing_count = jd_df.filter(
    col("required_education").isNull() |
    (trim(col("required_education")) == "")
).count()
req_ed_missing_pct   = (req_ed_missing_count / total_jd * 100) if total_jd else 0.0
log(f"\nTotal jd_df rows:                            {total_jd:,}")
log(f"required_education NULL or empty:            {req_ed_missing_count:,} ({req_ed_missing_pct:.2f}%)")

# 4) % of jd_df.certifications arrays with length < 3
jd_cert_lt3_count = jd_df.filter(size(col("certifications")) < 1).count()
jd_cert_lt3_pct   = (jd_cert_lt3_count / total_jd * 100) if total_jd else 0.0
log(f"certifications length < 3:                   {jd_cert_lt3_count:,} ({jd_cert_lt3_pct:.2f}%)")


Total resume_df rows:                        1,038
education length < 3:                        806 (77.65%)
certifications length == 0:                  801 (77.17%)

Total jd_df rows:                            1,038
required_education NULL or empty:            304 (29.29%)
certifications length < 3:                   722 (69.56%)


In [10]:
from pyspark.sql.functions import size, col

# Filter resume_df for education arrays of length < 3 and select relevant columns
filtered_df = resume_df.filter(size(col("education")) < 2) \
                       .select("id", "education")

# Convert to a pandas DataFrame
sample_df = filtered_df.toPandas()

# Optional: preview
print(sample_df.shape)
sample_df.head()


(354, 2)


,id,education
0,RES_mFb6f8iE,[]
1,RES_PqbOkeex,[]
2,RES_bJhBsOTi,"[(Associates of Applied Science Degree: Cyber Security, Chattahoochee Technical College, 2011-01-01T00:00:00, None, None, Security+ Certified Professional, Network+ Certified Professional, Server+ Certified Professional, A+ Certified Professional..."
3,RES_VGtTStBM,[]
4,RES_nSUQ70Ez,"[(Bachelor of Arts, HNBPG Govt Collage, None, 2016-05-01T00:00:00, None, Art, None, None)]"


In [5]:
# Cell 1 ─────────────────────────────────────────────────────────
from pyspark.sql import SparkSession

spark = (SparkSession.getActiveSession()
         or SparkSession.builder
             .appName("EduParse_NoEmbeddings")
             .getOrCreate())
spark.conf.set("spark.sql.debug.maxToStringFields", 400)

pq_path = "datamart/bronze/resume_2021-9.parquet"  # adjust if needed
df = (spark.read.parquet(pq_path)
      .select("id", "education", "certifications"))

print("Rows loaded →", df.count())
df.limit(2).show(truncate=False)


Rows loaded → 1027
+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+
|id          |education                                                                                                                                                                                                                                                           |certifications|
+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+
|RES_kUoPlNoI|[{Bachelor of Engineering, University of Pune, 2010-01-01T00:00:00, NULL, NULL, Electronics an

In [6]:
# Cell 2 ─────────────────────────────────────────────────────────
import json

# 1) pull into pandas (nested structs become lists of Row objects)
pdf = df.toPandas()

# 2) serialize the nested arrays as JSON strings so CSV can carry them end-to-end
pdf["education"] = pdf["education"]\
    .apply(lambda arr: json.dumps([r.asDict() for r in arr], default=str))
pdf["certifications"] = pdf["certifications"]\
    .apply(lambda arr: json.dumps(arr, default=str))

# 3) (optional) save out the Spark schema so you know how to deserialize later
with open("resume_schema.json", "w") as f:
    f.write(df.schema.json())

# 4) write to CSV
pdf.to_csv("resume_data.csv", index=False)
print("Wrote resume_data.csv with", len(pdf), "rows")


Wrote resume_data.csv with 1027 rows


# Start imputation


In [11]:
# Cell 1 ─────────────────────────────────────────────────────────
from pyspark.sql import SparkSession

spark = (SparkSession.getActiveSession()
         or SparkSession.builder.appName("EduParse_NoEmbeddings").getOrCreate())
spark.conf.set("spark.sql.debug.maxToStringFields", "400")

pq_path = "datamart/bronze/resume_2021-8.parquet"
df = (spark.read.parquet(pq_path)
      .select("id", "education", "certifications"))

print("Rows loaded →", df.count())
df.limit(2).show(truncate=False)


Rows loaded → 1038
+------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+
|id          |education                                                                                                                                                                                                                                                                                                                                                                                                                 

In [12]:
# Cell 2 ─────────────────────────────────────────────────────────
import os, textwrap, re
from pathlib import Path
Path("utils").mkdir(exist_ok=True)

with open("utils/edu_utils.py", "w") as fh:
    fh.write(textwrap.dedent(r"""
        # utils/edu_utils.py  – regex + rapid-fuzz only
        import re, unicodedata
        from rapidfuzz import process, fuzz

        # ─── helpers ────────────────────────────────────────────
        def _norm(t):
            if not t: return ""
            t = unicodedata.normalize("NFKD", t).encode("ascii","ignore").decode()
            return t.lower().strip()

        # ─── DEGREE-LEVEL synonyms ──────────────────────────────
        LEVEL_SYNONYMS = {
            # Doctorate
            "doctor": "doctorate", "doctorate":"doctorate", "phd":"doctorate",
            "ph.d":"doctorate", "dba":"doctorate", "phd of science":"doctorate",
            "juris doctor":"doctorate", "jd":"doctorate",
            # Master
            "master": "master", "masters": "master", "master degree":"master",
            "msc":"master", "m.sc":"master", "ms ":"master", "m.s":"master",
            "meng":"master", "m.eng":"master", "mba":"master", "m.b.a":"master",
            "macc":"master", "master of accountancy":"master",
            # Bachelor
            "bachelor":"bachelor", "bachelors":"bachelor", "bachelor's":"bachelor",
            "bs ":"bachelor", "b.s":"bachelor", "bsc":"bachelor", "b.sc":"bachelor",
            "ba ":"bachelor", "b.a":"bachelor", "beng":"bachelor", "b.eng":"bachelor",
            "beng (hons)":"bachelor", "b.tech":"bachelor", "btech":"bachelor",
            "be ":"bachelor", "b.e":"bachelor", "bba":"bachelor", "b.s.":"bachelor",
            # Associate
            "associate":"associate", "associates":"associate", "associate degree":"associate",
            "aa ":"associate", "a.a":"associate", "aas":"associate", "a.a.s":"associate",
            "as ":"associate", "a.s":"associate",
            # Diploma / certificate / course / secondary
            "post graduate diploma":"diploma", "pg diploma":"diploma", "diploma":"diploma",
            "technical diploma":"diploma", "certificate":"certificate",
            "certification":"certificate", "course on":"certificate",
            "license":"certificate",
            "high school":"secondary", "secondary school":"secondary",
            "ged":"secondary", "some college":"secondary"
        }
        _LEVEL_KEYS = list(LEVEL_SYNONYMS.keys())

        # ─── MAJOR list  (≈120) ─────────────────────────────────
        MAJOR_LIST = list({
            # engineering / stem
            "computer science","computer engineering","software engineering",
            "information technology","information systems","data science",
            "data analytics","business analytics","statistics","mathematics",
            "applied mathematics","electronics engineering",
            "electronics and communication engineering",
            "electrical engineering","electrical and computer engineering",
            "electrical electronics and communications engineering",
            "electronics & telecommunication","telecommunications engineering",
            "telecommunications","civil engineering","chemical engineering",
            "mechanical engineering","mechatronics","biomedical engineering",
            "biomedical diagnostics","biological sciences","biology",
            "healthcare administration","public health","health administration",
            "industrial engineering","industrial microbiology",
            "resource economics","applied economics",
            "environmental studies","environmental science",
            # business / social / other
            "business administration","business studies","finance",
            "accounting","commerce","management information systems",
            "management","marketing","economics","political science",
            "sociology","psychology","liberal arts","criminal justice",
            "law","human resource management","project management",
            "hospitality and tourism management","education",
            "electrical occupations","cosmetology","nursing","media arts"
        })

        GPA_RE = re.compile(r"(\\d+\\.\\d{1,2}|\\d)(?=\\s*/?\\s*10?\\.?0?)")

        # ─── extraction helpers ─────────────────────────────────
        def level_from_text(txt:str):
            low = _norm(txt)
            for k,v in LEVEL_SYNONYMS.items():
                if k in low: return v, "regex"
            m,score,_ = process.extractOne(low, _LEVEL_KEYS, scorer=fuzz.partial_ratio)
            return (LEVEL_SYNONYMS[m],"fuzzy") if score>=88 else (None,None)

        def major_from_text(txt:str):
            low = _norm(txt)
            for m in MAJOR_LIST:
                if m in low: return m, "regex"
            m2,score,_ = process.extractOne(low, MAJOR_LIST, scorer=fuzz.partial_ratio)
            return (m2,"fuzzy") if score>=85 else (None,None)

        def gpa_from_text(txt:str):
            m = GPA_RE.search(txt or "")
            return float(m.group()) if m else None
    """))
print("utils/edu_utils.py rebuilt with extended dictionaries ✅")
spark.sparkContext.addPyFile("utils/edu_utils.py")


utils/edu_utils.py rebuilt with extended dictionaries ✅


25/06/16 18:50:46 WARN SparkContext: The path utils/edu_utils.py has been added already. Overwriting of added paths is not supported in the current version.


In [13]:
# Cell 3 ─────────────────────────────────────────────────────────
from utils.edu_utils import level_from_text, major_from_text, gpa_from_text
from pyspark.sql.types import *
from pyspark.sql import Row

SCHEMA = StructType([
    StructField("id",             StringType(), False),
    StructField("highest_level",  StringType(), True),
    StructField("major",          StringType(), True),
    StructField("gpa",            FloatType(),  True),
    StructField("institution",    StringType(), True),
    StructField("level_strategy", StringType(), True),
    StructField("major_strategy", StringType(), True),
    StructField("certifications", ArrayType(StringType()), True),
])

total_rows = df.count()

def parse_partition(it):
    for idx,row in enumerate(it,1):
        if idx==1 or idx%1000==0:
            print(f"[partition] processed {idx} rows …")
        edu = row.education
        if not edu:
            yield Row(id=row.id, highest_level=None, major=None, gpa=None,
                      institution=None, level_strategy=None, major_strategy=None,
                      certifications=row.certifications)
            continue

        ed0 = edu[0].asDict()
        combo = f"{ed0.get('degree','')} {ed0.get('description','')}"
        lvl,lvl_s = level_from_text(combo)
        maj,maj_s = major_from_text(combo)
        gpa = ed0.get("grade") or gpa_from_text(combo)

        yield Row(id=row.id, highest_level=lvl, major=maj,
                  gpa=float(gpa) if gpa else None,
                  institution=ed0.get("institution"),
                  level_strategy=lvl_s, major_strategy=maj_s,
                  certifications=row.certifications)

parsed_df = spark.createDataFrame(df.rdd.mapPartitions(parse_partition),
                                  SCHEMA)

print("⇢ quick sample")
parsed_df.select("id","highest_level","major","gpa","institution",
                 "level_strategy","major_strategy").show(12, truncate=False)


⇢ quick sample
+------------+-------------+--------------------+----+----------------------------------------------------------------------+--------------+--------------+
|id          |highest_level|major               |gpa |institution                                                           |level_strategy|major_strategy|
+------------+-------------+--------------------+----+----------------------------------------------------------------------+--------------+--------------+
|RES_CHUA6p90|bachelor     |public health       |3.65|California State University Northridge                                |regex         |regex         |
|RES_SlLMkiqd|doctorate    |public health       |NULL|George Washington University, Milken Institute School of Public Health|regex         |regex         |
|RES_mFb6f8iE|NULL         |NULL                |NULL|NULL                                                                  |NULL          |NULL          |
|RES_zkuNDo8d|bachelor     |computer science    |

[partition] processed 1 rows …
[partition] processed 1000 rows …


In [14]:
# Cell 4 ─────────────────────────────────────────────────────────
out_csv = "resume_edu_extracted_2021-08_noEmbeddings.csv"
parsed_df.toPandas().to_csv(out_csv, index=False)
print("✅ CSV exported →", out_csv)


✅ CSV exported → resume_edu_extracted_2021-08_noEmbeddings.csv


[partition] processed 1 rows …
[partition] processed 1000 rows …


In [ ]:
print('gimpo')

In [36]:
# # ╔══════════════════════════════════════════════════════════════════╗
# # ║  Cell – Save silver-level education tables to single-file CSVs   ║
# # ╚══════════════════════════════════════════════════════════════════╝

# import os, glob, shutil

# # -------------------------------------------------------------------
# # OPTION A – small/medium data → collect to Pandas (fast & simple)
# # -------------------------------------------------------------------
# # Uncomment if each DataFrame fits comfortably in driver memory.
# #
# # res_edu_imp.toPandas().to_csv("resume_edu_silver_2021-08.csv", index=False)
# # jd_edu     .toPandas().to_csv("jd_edu_silver_2021-08.csv",     index=False)
# # print("✓ CSVs written (Pandas collect).")
# #
# # -------------------------------------------------------------------
# # OPTION B – large data → distributed write, then re-name single part
# # -------------------------------------------------------------------

# def save_single_csv(df, out_name):
#     tmp_dir = out_name + "_tmp"
#     # coalesce(1) → exactly one part file
#     df.coalesce(1).write.csv(tmp_dir, header=True, mode="overwrite")
#     part_file = glob.glob(f"{tmp_dir}/part-*")[0]
#     shutil.move(part_file, out_name)
#     shutil.rmtree(tmp_dir)
#     print(f"✓ {out_name}  ({df.count():,} rows)")

# save_single_csv(res_edu_imp, "resume_edu_silver_2021-08.csv")
# save_single_csv(jd_edu,      "jd_edu_silver_2021-08.csv")


In [1]:
import os, glob
from pyspark.sql import SparkSession

# ── Spark session (reuse the one in your notebook / script if it exists) ─────────
spark = (SparkSession.builder
         .appName("Parquet→CSV export")
         .getOrCreate())

# ── I/O directories ─────────────────────────────────────────────────────────────
base_dir      = os.path.join(os.getcwd(), "datamart")
silver_dir    = os.path.join(base_dir, "silver")
csv_out_dir   = os.path.join(base_dir, "csv_files")
os.makedirs(csv_out_dir, exist_ok=True)

# ── grab all silver parquet paths we just generated ─────────────────────────────
parquet_paths = sorted(
    glob.glob(os.path.join(silver_dir, "resume_silver_2021-*.parquet"))
)

print(f"Found {len(parquet_paths)} parquet files")

# ── loop & export ───────────────────────────────────────────────────────────────
for pq_path in parquet_paths:
    fname  = os.path.basename(pq_path).replace(".parquet", ".csv")
    csv_fp = os.path.join(csv_out_dir, fname)

    # Read with Spark
    pdf = spark.read.parquet(pq_path).toPandas()

    # Persist to CSV
    pdf.to_csv(csv_fp, index=False)
    print(f"✓ wrote {csv_fp}  rows={len(pdf)}")

print("All done!")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/16 20:13:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Found 12 parquet files
✓ wrote /app/datamart/csv_files/resume_silver_2021-1.csv  rows=0
✓ wrote /app/datamart/csv_files/resume_silver_2021-10.csv  rows=1016
✓ wrote /app/datamart/csv_files/resume_silver_2021-11.csv  rows=996
✓ wrote /app/datamart/csv_files/resume_silver_2021-12.csv  rows=0
✓ wrote /app/datamart/csv_files/resume_silver_2021-2.csv  rows=0
✓ wrote /app/datamart/csv_files/resume_silver_2021-3.csv  rows=0
✓ wrote /app/datamart/csv_files/resume_silver_2021-4.csv  rows=0
✓ wrote /app/datamart/csv_files/resume_silver_2021-5.csv  rows=0
✓ wrote /app/datamart/csv_files/resume_silver_2021-6.csv  rows=1062
✓ wrote /app/datamart/csv_files/resume_silver_2021-7.csv  rows=1102
✓ wrote /app/datamart/csv_files/resume_silver_2021-8.csv  rows=1038
✓ wrote /app/datamart/csv_files/resume_silver_2021-9.csv  rows=1027
All done!
